# 使用TFLite导出模型

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from sklearn import datasets
import numpy as np
import os

In [2]:
x_train = datasets.load_iris().data
y_train = datasets.load_iris().target

In [3]:
np.random.seed(116)
np.random.shuffle(x_train)
np.random.seed(116)
np.random.shuffle(y_train)
tf.random.set_seed(116)

In [4]:
# 定义网络结构
class IrisModel(Model):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.d1 = Dense(10, activation='relu', kernel_regularizer=tf.keras.regularizers.l2())
        self.d1 = Dense(10, activation='relu', kernel_regularizer=tf.keras.regularizers.l2())
        self.d2 = Dense(3, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2())

    def call(self, x):
        y = self.d1(x)
        y = self.d2(x)
        return y
# 实例化化模型
model = IrisModel()

In [5]:
# 初始化模型
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=300, validation_split=0.2, validation_freq=20)
model.summary()

D:\PC\Anaconda\envs\tensorflow2.6\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/300
4/4 [==============================] - 1s 5ms/step - loss: 5.0269 - sparse_categorical_accuracy: 0.3000
Epoch 2/300
4/4 [==============================] - 0s 0s/step - loss: 1.0074 - sparse_categorical_accuracy: 0.6750
Epoch 3/300
4/4 [==============================] - 0s 5ms/step - loss: 0.8310 - sparse_categorical_accuracy: 0.7333
Epoch 4/300
4/4 [==============================] - 0s 0s/step - loss: 0.7466 - sparse_categorical_accuracy: 0.7750
Epoch 5/300
4/4 [==============================] - 0s 0s/step - loss: 0.9348 - sparse_categorical_accuracy: 0.6583
Epoch 6/300
4/4 [==============================] - 0s 5ms/step - loss: 0.7391 - sparse_categorical_accuracy: 0.7583
Epoch 7/300
4/4 [==============================] - 0s 0s/step - loss: 0.6323 - sparse_categorical_accuracy: 0.7583
Epoch 8/300
4/4 [==============================] - 0s 0s/step - loss: 0.6005 - sparse_categorical_accuracy: 0.8417
Epoch 9/300
4/4 [==============================] - 0s 5ms/step - loss: 0.6061

## 将模型保存为tflite格式

In [9]:
tflite_save_path = './export/0_example'
os.makedirs(tflite_save_path, exist_ok=True)
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Save the model.
with open(tflite_save_path+'/model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DAIXIN~1\AppData\Local\Temp\tmpxcli_j0e\assets


INFO:tensorflow:Assets written to: C:\Users\DAIXIN~1\AppData\Local\Temp\tmpxcli_j0e\assets


## 加载tflite格式模型并进行推理测试

In [12]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_save_path+'/model.tflite')  # 加载模型
interpreter.allocate_tensors()  # 为模型分配张量参数

# Get input and output tensors.
input_details = interpreter.get_input_details()     # 设置网络输入
output_details = interpreter.get_output_details()   # 设置网络输出

# Test the model on set input data.
input_shape = input_details[0]['shape'] # 获取输入层（第一层）的数据维度
print(input_shape)

input_data = np.array([6.0,3.4,4.5,1.3], dtype=np.float32)
input_data = input_data.reshape([1,4])  # 确保维度相同
print(input_data.shape)
interpreter.set_tensor(input_details[0]['index'], input_data)   # 将数据输入到网络中
interpreter.invoke()  # 运行推理
output_data = interpreter.get_tensor(output_details[0]['index'])    # 获得网络输出

print(output_data)

pred = tf.argmax(output_data, axis=1)   # 网络输出层是softmax，需要找到最大值
print(int(pred))    # 输出最大位置的index


[1 4]
(1, 4)
[[0.04466662 0.8185583  0.13677515]]
1
